In [38]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from preprocess import preprocess_image_tf, preprocess_image

os.environ['CUDA_VISIBLE_DEVICES'] = '0'




In [52]:
df = pd.DataFrame(columns=['file', 'label', 'image'])

data_path = 'data2/'

for disease in os.listdir(data_path):
    for filename in os.listdir(os.path.join(data_path, disease)):
        df_dictionary = pd.DataFrame([{'file': os.path.join(data_path, disease, filename), 'label': disease}])
        df = pd.concat([df, df_dictionary], ignore_index=True)

df['image'] = None


for index, row in df.iterrows():
    proccessed_image = preprocess_image(row['file'])
    df.loc[index, 'image'] = proccessed_image


df.drop('file', axis=1, inplace=True)


In [53]:
df['image'][0].dtype, df['image'][0].shape

(dtype('float32'), (224, 224, 3))

In [54]:

# Define the input shape of the model
input_shape = (224, 224, 3)

# Create a Sequential model
model = tf.keras.Sequential()

# Add a convolutional layer with 32 filters, 3x3 kernel size, and ReLU activation
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))

# Add a convolutional layer with 64 filters, 3x3 kernel size, and ReLU activation
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer with 2x2 pool size
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add a flattening layer
model.add(tf.keras.layers.Flatten())

# Add a dense layer with 128 units and ReLU activation
model.add(tf.keras.layers.Dense(128, activation='relu'))

# Add a dense output layer with the same number of units as the number of unique labels in df
model.add(tf.keras.layers.Dense(len(df['label'].unique()), activation='softmax'))


In [63]:

trainingdataset, validatedataset = train_test_split(df, test_size=0.2, random_state=42) 

# print(df['image'][0].dtype)

# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

valid_ds = df.take(n_samples)
train_ds = df.skip(n_samples)


float32
(368,)
(92,)
(368,)
(92,)


In [66]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])   

model.fit(train_ds, validation_data=valid_ds, batch_size=32, epochs=10)

# model.save('model.h5')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).